1+1

In [98]:
] build FFMPEG

  Building FFMPEG → `~/.julia/packages/FFMPEG/bkWgb/deps/build.log`


In [99]:
using Pkg
using LinearAlgebra
using SparseArrays

Pkg.add("Plots")
using Plots
Pkg.add("BenchmarkTools")
using BenchmarkTools

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [100]:
function make_matrix(N,alpha_thermal)
    h=1/(N-1)
    dl = -1/h*ones(N-1);
    du = -1/h*ones(N-1);
    d = 1/h*2*ones(N);
    
    d[1]=1/h+alpha_thermal
    d[N]=1/h+alpha_thermal
    A=Tridiagonal(dl, d, du)
    return A
end

make_matrix (generic function with 1 method)

In [101]:
function Progonka(A,f)
    N=length(f)
    u=zeros(N)
    a=[0;diag(A[2:end,1:end-1])]
    b=diag(A)
    c=diag(A[1:end-1,2:end])
    alpha=zeros(N)
    beta=zeros(N)
    
    alpha[2]=-c[1]/b[1]
    beta[2]=f[1]/b[1]
    
    for i in 2:N-1
        alpha[i+1]=-c[i]/(a[i]*alpha[i]+b[i])
        beta[i+1]=(f[i]-a[i]*beta[i])/(a[i]*alpha[i]+b[i])
    end
    u[N]=(f[N]-a[N]*beta[N])/(a[N]*alpha[N]+b[N])
    for i in N-1:-1:1
        u[i]=alpha[i+1]*u[i+1]+beta[i+1]
    end
    return u
end 

Progonka (generic function with 1 method)

In [102]:
k=6
alpha_thermal=1
v_l=0
v_r=0

N=2^k+1
h=1/(N-1)
u_real=-h^2/2*collect(1:N).*collect(1:N) .+ h/2*collect(1:N) .+ 1/2
f=h*ones(N)
f[1]=alpha_thermal*v_l
f[N]=alpha_thermal*v_r

0

In [103]:
A=make_matrix(N,alpha_thermal)

4097×4097 Tridiagonal{Float64,Array{Float64,1}}:
  4097.0  -4096.0       ⋅        ⋅   …       ⋅        ⋅        ⋅        ⋅ 
 -4096.0   8192.0  -4096.0       ⋅           ⋅        ⋅        ⋅        ⋅ 
      ⋅   -4096.0   8192.0  -4096.0          ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅   -4096.0   8192.0          ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅   -4096.0          ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅   …       ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅   …       ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
      ⋅        ⋅        ⋅        ⋅           ⋅     

In [104]:
@show error1=norm(u_real-Progonka(A,f),1)/N
@benchmark Progonka(A,f)

error1 = norm(u_real - Progonka(A, f), 1) / N = 0.00012204051645344211


BenchmarkTools.Trial: 
  memory estimate:  836.49 KiB
  allocs estimate:  144
  --------------
  minimum time:     436.586 ms (0.00% GC)
  median time:      458.740 ms (0.00% GC)
  mean time:        455.775 ms (0.10% GC)
  maximum time:     471.262 ms (1.09% GC)
  --------------
  samples:          11
  evals/sample:     1

In [105]:
@show error2=norm(u_real-A\f,1)/N
@benchmark A\f

error2 = norm(u_real - A \ f, 1) / N = 0.0001220405164561813


BenchmarkTools.Trial: 
  memory estimate:  176.73 KiB
  allocs estimate:  15
  --------------
  minimum time:     119.639 μs (0.00% GC)
  median time:      127.528 μs (0.00% GC)
  mean time:        207.612 μs (30.33% GC)
  maximum time:     4.681 ms (93.26% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [106]:
A=sparse(A)
@show error3=norm(u_real-A\f,1)/N
@benchmark A\f

error3 = norm(u_real - A \ f, 1) / N = 0.00012204051240483864


BenchmarkTools.Trial: 
  memory estimate:  1.08 MiB
  allocs estimate:  49
  --------------
  minimum time:     1.340 ms (0.00% GC)
  median time:      1.509 ms (0.00% GC)
  mean time:        2.050 ms (18.87% GC)
  maximum time:     7.287 ms (61.09% GC)
  --------------
  samples:          2425
  evals/sample:     1

In [107]:
A=Matrix(A)
@show error4=norm(u_real-A\f,1)/N
@benchmark A\f

error4 = norm(u_real - A \ f, 1) / N = 0.0001220405169308017


InexactError: InexactError: trunc(Int32, 3.203524328e9)

In [108]:
@show error5=norm(u_real-inv(A)*f,1)/N
@benchmark inv(A)*f

x=h*collect(1:N)
plot(x,u_real)
plot!(x,inv(A)*f)

error5 = norm(u_real - inv(A) * f, 1) / N = 0.00012204051693118792


InexactError: InexactError: trunc(Int32, 1.0664323998e10)

InexactError: InexactError: trunc(Int32, 1.1536691816e10)

1025-element Array{Float64,1}:
 0.499511718750261  
 0.49999952316310453
 0.5004863739016275 
 0.50097227096584   
 0.5014572143557301 
 0.5019412040713059 
 0.5024242401125699 
 0.5029063224795127 
 0.503387451172136  
 0.5038676261904517 
 0.5043468475344451 
 0.5048251152041197 
 0.5053024291994862 
 ⋮                  
 0.504825115204396  
 0.5043468475347178 
 0.5038676261907236 
 0.5033874511724121 
 0.502906322479785  
 0.5024242401128413 
 0.5019412040715807 
 0.5014572143560043 
 0.5009722709661111 
 0.5004863739019014 
 0.49999952316337554
 0.49951171875053346